# Clustering 

(Run this code after you have run the matlab code "clustering_meta_k_means.m")

### 1) Spatial distribution of neuron clusters
### 2) Correlation coefficients between cluster centroids
### 3) Spatial correlations between cluster centroids
### 4) Correlations between individual neurons' activity patterns
### 5) Spectral Analysis


## Setting up environment and defining paths

In [450]:
import h5py
import scipy.sparse as sp
from scipy.io import loadmat
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr, iqr
from scipy.signal import welch
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FormatStrFormatter
import pandas as pd
import numpy as np
import pywt

import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_1453689/82211235.py:34: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
/tmp/ipykernel_1453689/82211235.py:35: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')


Loading BokehJS ...

### Only things you need to define are: organoid_number, run, and save_figure.

In [492]:
# Define your variables
organoid_number = 'organoid1'
run = 'run_2'

# Define 1 to save the figure, 0 otherwise
save_figure = 1

In [493]:
# Construct your paths using string formatting
base_path1 = '/home/silviu/erika-organoid-data/CODE/CURRENT-FOLDER/LOOP/exp1-2_11-25_07_23'
base_path2 = '/home/silviu/erika-organoid-data/CODE/CURRENT-FOLDER/Clustering-Correlation-Distance/loop-new'

path_analysis_results = f'{base_path1}/{organoid_number}/{run}' # Contains the "analysis_results.hdf5" file which is the ouput of CaIman
path_clustering_data = f'{base_path2}/{organoid_number}/{run}' # Contains the "clustering_data.mat" file which is the output of the "meta_k_means" code

# Now, path_analysis_results and path_clustering_data are constructed from the variables
print("Path to Analysis Results:", path_analysis_results)
print("Path to Clustering Data:", path_clustering_data)

Path to Analysis Results: /home/silviu/erika-organoid-data/CODE/CURRENT-FOLDER/LOOP/exp1-2_11-25_07_23/organoid1/run_2
Path to Clustering Data: /home/silviu/erika-organoid-data/CODE/CURRENT-FOLDER/Clustering-Correlation-Distance/loop-new/organoid1/run_2


In [494]:
# For manually setting the paths
# path_analysis_results = '/home/silviu/erika-organoid-data/CODE/CURRENT-FOLDER/LOOP/exp1-2_11-25_07_23/organoid4/run'
# path_clustering_data = '/home/silviu/erika-organoid-data/CODE/CURRENT-FOLDER/Clustering-Correlation-Distance/loop-new/organoid4/run'

In [495]:
# Conversion ratios from pixels to MICROMETERS for different organoids
# Depends on the microscope magnification used for each organoid's recordings
conversion_ratios = {'organoid7': 10.6,   # 94 pixels = 1 mm
                     'organoid1': 6.58,   # 152 pixels = 1 mm
                     'organoid2': 6.58,   # 152 pixels = 1 mm
                     'organoid6': 6.58,   # 152 pixels = 1 mm
                     'organoid5': 5.26,   # 190 pixels = 1 mm
                     'organoid3': 4.08,   # 245 pixels = 1 mm
                     'organoid4': 4.08,   # 245 pixels = 1 mm
                     'organoid8': 4.08,   # 245 pixels = 1 mm
                     'organoid9': 4.08}   # 245 pixels = 1 mm

# Determine the conversion factor based on the organoid_number
conversion_factor = conversion_ratios.get(organoid_number, None)
if conversion_factor is None:
    raise ValueError(f"No conversion ratio defined for organoid number {organoid_number}")

## 1) Spatial distribution of neuron clusters

In [ ]:
# Automatically find the 'analysis_results.hdf5' and the largest '.mmap' file in the specified directory
analysis_file = os.path.join(path_analysis_results, 'analysis_results.hdf5')

mmap_files = [f for f in os.listdir(path_analysis_results) if f.endswith('.mmap')]
if not mmap_files:
    raise FileNotFoundError("No .mmap files found in the specified directory.")
largest_mmap_file = max(mmap_files, key=lambda f: os.path.getsize(os.path.join(path_analysis_results, f)))
mmap_file_path = os.path.join(path_analysis_results, largest_mmap_file)

# Load the data from the HDF5 file
with h5py.File(analysis_file, 'r') as f:
    estimates = f['estimates']
    data = estimates['A']['data'][:]
    indices = estimates['A']['indices'][:]
    indptr = estimates['A']['indptr'][:]
    shape = estimates['A']['shape'][:]
    A = sp.csc_matrix((data, indices, indptr), shape=shape)
    C = np.array(estimates['C'])
    dims = np.array(estimates['dims'])

# Load the memory-mapped file to calculate the correlation image
Yr, dims, T = cm.load_memmap(mmap_file_path)
images = np.reshape(Yr.T, [T] + list(dims), order='F')
Cn = cm.local_correlations(images.transpose(1, 2, 0))
Cn[np.isnan(Cn)] = 0

# Create a CNMF object
cnm = cnmf.CNMF(n_processes=None, dview=None)

# Create an Estimates object and populate with loaded results
cnm.estimates = cnmf.Estimates()
cnm.estimates.A = A
cnm.estimates.C = C
cnm.estimates.dims = dims

# Automatic contrast adjustment using Histogram Equalization
Cn = cv2.normalize(Cn, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
Cn = cv2.equalizeHist((Cn * 255).astype(np.uint8)).astype(np.float32) / 255

# Use native CaImAn functions for plotting
cnm.estimates.plot_contours_nb(img=Cn)

Decode mmap filename /home/silviu/erika-organoid-data/CODE/CURRENT-FOLDER/LOOP/exp1-2_11-25_07_23/organoid1/run_2/memmap__d1_540_d2_640_d3_1_order_C_frames_12714.mmap


In [ ]:
# Extract the centroids of the neurons
centroids = np.zeros((A.shape[1], 2))  # Assuming A is Neurons x Pixels
for i, a in enumerate(A.T):
    spatial_footprint = a.toarray().reshape(*dims, order='F')
    y, x = np.indices(dims)
    total_intensity = spatial_footprint.sum()
    centroids[i, 0] = (x * spatial_footprint).sum() / total_intensity  # x-coordinate (column index)
    centroids[i, 1] = (y * spatial_footprint).sum() / total_intensity  # y-coordinate (row index)

In [ ]:
# centroids

In [ ]:
# Load the 'clustering_data.mat' from the specified directory
# matlab_data_path = os.path.join(path_clustering_data, 'clustering_data.mat')
matlab_data_path = os.path.join(path_clustering_data, 'clustering_data_new.mat') # trying with the "new"
matlab_data = loadmat(matlab_data_path)

allclusters_new = matlab_data['allclusters_new']
# allclusters = matlab_data['allclusters'] # Contains the neuron indexes that correspond to clusters (this is a cell array containing the final clusters (with the neuron indexes that correspond to it) and their centroids)
remove_columns = matlab_data['remove_columns'] # Contains 1's for the columns that got deleted

In [ ]:
# np.save('/home/silviu/Downloads/matlab_data.npy', matlab_data)

In [ ]:
# Check that the length of the 'remove_columns' matches the number of entries in the centroids array to ensure they are in sync
if centroids.shape[0] != remove_columns.shape[1]:
    raise ValueError("Length of centroids and remove_columns do not match!")

In [ ]:
# Use the 'remove_columns' variable to filter out entries from the centroids array
filtered_centroids = centroids[remove_columns.flatten() == 0]

In [ ]:
# np.save('/home/silviu/Downloads/filtered_centroids.npy', filtered_centroids)

In [ ]:
# Extract and process the allclusters matrix
neuron_indices = [row[0].flatten() for row in matlab_data['allclusters_new'][:, 0]]
cluster_ids = list(range(1, len(neuron_indices) + 1))

neuron_to_cluster = {}
for cluster_id, indices in zip(cluster_ids, neuron_indices):
    for idx in indices:
        # Only assign the neuron to a cluster if it hasn't been assigned yet
        if idx not in neuron_to_cluster:
            neuron_to_cluster[idx] = cluster_id

In [ ]:
# neuron_to_cluster

In [ ]:
# Generate cluster assignments for each centroid based on the neuron_to_cluster mapping
cluster_assignments = np.array([neuron_to_cluster.get(i+1, 0) for i in range(len(filtered_centroids))])

# Stack centroids with their corresponding cluster assignments
labeled_neurons = np.column_stack((filtered_centroids, cluster_assignments))

In [ ]:
# labeled_neurons

In [ ]:
# np.save('/home/silviu/Downloads/labeled_neurons.npy', labeled_neurons)

In [ ]:
# Get the unique cluster indices
unique_clusters = np.unique(labeled_neurons[:, 2])

# Define a custom colormap
darker_grey = (0.3, 0.3, 0.3)
colors = [darker_grey] + [plt.cm.jet(i) for i in range(plt.cm.jet.N)]
custom_colormap = mcolors.LinearSegmentedColormap.from_list('custom_colormap', colors, N=len(colors))

# White edge around the grey colored cluster "not assigned"
colors_array = [custom_colormap(cluster / (len(unique_clusters) - 1 if cluster != 0 else 1))
                for cluster in labeled_neurons[:, 2]]
# Only white edges:
# edge_colors = ['white' if cluster == 0 else color for cluster, color in zip(labeled_neurons[:, 2], colors_array)]
# White and black edges around the specified clusters:
edge_colors = ['white' if cluster == 0 else 'black' for cluster in labeled_neurons[:, 2]]

# Plotting
plt.figure(figsize=(10, 8))

# Display the flipped background image
flipped_Cn = np.flipud(Cn)
plt.imshow(flipped_Cn, cmap='gray', extent=[0, dims[1], dims[0], 0])  # extent sets the limits of the image in the plot

# Extract original coordinates
original_x = labeled_neurons[:, 1]
original_y = labeled_neurons[:, 0]

# Apply cumulative transformations:
# Two 90 degree clockwise rotations and two mirrorings over X-axis
final_x = dims[0] - original_x  # Mirroring over X-axis after rotations
final_y = original_y  # Y-coordinates remain the same after rotations and mirroring over X-axis

# Overlay the final scatter plot
# Without white edges, just make sure to comment out the code further above that does that...
# scatter = plt.scatter(final_y, final_x, c=labeled_neurons[:, 2], cmap=custom_colormap, s=100, alpha=0.6)
# With white edges, but no black edges around the nuerons that were clustered
# scatter = plt.scatter(final_y, final_x, c=labeled_neurons[:, 2], cmap=custom_colormap, s=100, alpha=0.6, edgecolor=edge_colors)
scatter = plt.scatter(final_y, final_x, c=labeled_neurons[:, 2], cmap=custom_colormap, s=100, alpha=0.6, edgecolor=edge_colors) 


# Create legend
adjusted_labels = ['Not Assigned' if cluster == 0 else f'Cluster {int(cluster)}' for cluster in unique_clusters]
legend_patches = [mpatches.Patch(color=custom_colormap(cluster / (len(unique_clusters) - 1 if cluster != 0 else 1)), label=label) 
                  for cluster, label in sorted(zip(unique_clusters, adjusted_labels))]
plt.legend(handles=legend_patches, loc='upper left')

plt.title('Spatial Distribution of Neuron Clusters')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

# Adjust the axis limits
plt.xlim(0, 640)
plt.ylim(0, 540)

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Spatial-Dist-Neuron-Clusters-new.png'))

plt.show()

#### Rarely happens but if the plot above messes up the legend of color assignments, try this code:
Sometimes happens for cases with datasets where ALL neurons were assigned to a cluster.
The usual case is that there are one or two neurons that do not get assigned to a cluster.

## 2) Correlation coefficients between cluster centroids

This code generates a heatmap to visualize the pairwise correlation coefficients between the centroids of clusters obtained from your data. The correlation coefficient provides a measure of how similar the activity patterns of two centroids are, with values close to 1 indicating strong similarity and values close to -1 indicating dissimilarity.

As you can see, the calculations are based solely on the centroids from the allclusters variable, without any reference to the original data in F_dff_dec. So this code is usable for all runs.

In [ ]:
# Extract all the centroids from the 'allclusters' variable
num_clusters = allclusters_new.shape[0]
centroids_from_data = [allclusters_new[i, 1] for i in range(num_clusters)]

# Convert centroids list to a numpy array
centroids_array = np.vstack(centroids_from_data) # vertically stacking the list of centroids to form a matrix where each row is a centroid

# Compute the pairwise correlation coefficients between centroids
centroid_correlations = np.corrcoef(centroids_array)

# Create a new figure with a specific size
plt.figure(figsize=(10, 8))

# Display the correlation matrix as an image
cax = plt.imshow(centroid_correlations, cmap="coolwarm", vmin=-1, vmax=1)

# Add a color bar to the side
cbar = plt.colorbar(cax, shrink=0.75)
cbar.set_label('Correlation Coefficient', rotation=270, labelpad=20)

# Annotate the heatmap with the correlation values
for i in range(centroid_correlations.shape[0]):
    for j in range(centroid_correlations.shape[1]):
        plt.text(j, i, "{:.2f}".format(centroid_correlations[i, j]),
                 ha='center', va='center', color='black' if abs(centroid_correlations[i, j]) < 0.5 else 'white')

# Adjust the appearance of the plot
plt.title("Pairwise Correlation Coefficients Between Cluster Centroids")
plt.xlabel("Cluster Centroid Index")
plt.ylabel("Cluster Centroid Index")
plt.xticks(np.arange(centroid_correlations.shape[0]), np.arange(1, centroid_correlations.shape[0] + 1))
plt.yticks(np.arange(centroid_correlations.shape[1]), np.arange(1, centroid_correlations.shape[1] + 1))
plt.tight_layout()

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Corrcoeffs-btw-Cluster-Centroids-new.png'))
    
plt.show()


## 3) Spatial correlations between cluster centroids

Correlations between Cluster Centroids: Here, you're looking at the average or representative activity pattern of each cluster (the centroid) and computing correlations based on these. This provides a higher-level view, focusing on the relationships between groups of neurons (clusters) rather than individual neurons.

If you're  interested in understanding how clusters of neurons (groups of neurons with similar activity) are spatially organized with respect to each other, then using the correlations between cluster centroids would be the right approach.

**********************************

Correlations between Individual Neurons' Activity Patterns (see section #4): This approach considers the activity patterns of every single neuron in the field of view. When you compute pairwise correlations based on these, you're assessing how similar each neuron's activity is to every other neuron's activity. This gives a very detailed view of the relationships between all neurons.

For the specific goal of understanding how the mean correlation between neurons changes as a function of their spatial distance, using the pairwise correlations between individual neurons' activity patterns would be more appropriate. This is because you want to understand the spatial organization of individual neurons based on their activity patterns.

If clusters with similar activity patterns (i.e., high correlation) tend to be closer together, the curve will show higher correlations for shorter distances. Conversely, if similar activity patterns are distributed throughout the field of view, the curve will be flatter. This can help elucidate any spatial structure or organization in the neuronal activity of the observed area.

## 4) Correlations between individual neurons' activity patterns
Identifying whether neurons that are closer together tend to have more strongly correlated activity patterns.
Plots show standard error bars NOT standard deviation.

In [ ]:
# Bin size is manually set to 150 micrometers

# Load deconvolved fluorescence signal data depending on the condition being analyzed
if run == 'run':
    num_rows = matlab_data['F_dff_dec'].shape[0]  # Get the number of rows
    neuronal_signal_data = matlab_data['F_dff_dec'][0:min(6600, num_rows), :] # takes only the first 5min 30sec of spont. activity, as standardized in our experimental scheme 
elif run in ['run_1', 'run_2', 'run_3']:
    neuronal_signal_data = matlab_data['F_dff_dec_segmented']
elif run in ['run_4', 'run_5']:
    neuronal_signal_data = matlab_data['F_dff_dec_subset']
else:
    print("Invalid run specified")
    neuronal_signal_data = None  # or some default value

# Calculate pairwise correlations
num_neurons = neuronal_signal_data.shape[1]
correlation_matrix = np.zeros((num_neurons, num_neurons))
for i in range(num_neurons):
    for j in range(i+1, num_neurons):
        corr, _ = pearsonr(neuronal_signal_data[:, i], neuronal_signal_data[:, j])
        correlation_matrix[i, j] = corr
        correlation_matrix[j, i] = corr

# Calculate pairwise Euclidean distances between neurons
spatial_coordinates = labeled_neurons[:, :2]
distances_matrix = squareform(pdist(spatial_coordinates)) * conversion_factor # converting distances to micrometers

# Define a fixed bin size in micrometers
bin_size_micrometers = 150  # Adjust as needed

# Calculate the maximum number of bins needed based on the maximum distance
max_distance = np.max(distances_matrix)
num_bins = int(np.ceil(max_distance / bin_size_micrometers))

# Create the bins
distance_bins = np.arange(0, bin_size_micrometers * (num_bins + 1), bin_size_micrometers)

distance_bin_indices = np.digitize(distances_matrix, distance_bins)

mean_correlations_per_bin = []
stderr_correlations_per_bin = []  # Use standard error (not standard deviation)

for i in range(1, len(distance_bins)):
    indices = np.where(distance_bin_indices == i)
    mean_corr = np.mean(correlation_matrix[indices])
    
    # Number of observations in the bin
    num_observations = len(correlation_matrix[indices])
    
    # Calculate standard deviation
    std_dev_corr = np.std(correlation_matrix[indices])
    
    # Calculate standard error: standard deviation divided by the square root of the number of observations
    if num_observations > 0:
        stderr_corr = std_dev_corr / np.sqrt(num_observations)
    else:
        stderr_corr = np.nan  # Assign NaN if no observations in the bin

    mean_correlations_per_bin.append(mean_corr)
    stderr_correlations_per_bin.append(stderr_corr)

# Plotting FULL signal
bins_center = (distance_bins[:-1] + distance_bins[1:]) / 2 # The bin centers will now also be consistently spaced according to the fixed bin size
plt.figure(figsize=(10, 6))
plt.errorbar(bins_center, mean_correlations_per_bin, yerr=stderr_correlations_per_bin, fmt='o', ecolor='darkgray', elinewidth=3, capsize=5, linestyle='-', linewidth=2)
plt.title('Mean Correlation vs. Spatial Distance Between Neurons \n (using equally spaced binning)')
plt.xlabel('Distance (μm)')
plt.ylabel('Mean Correlation Coefficient')
plt.grid(True)

# Annotate bin size in micrometers
bin_size_micrometers = (bins_center[1] - bins_center[0])
bin_size_micrometers = max(round(bin_size_micrometers), 1) # round to nearest whole number
micrometer_label = "μm" if bin_size_micrometers <= 1 else "μm"
plt.annotate(f'Bin Size: {round(bin_size_micrometers, 2)} {micrometer_label}', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=10,
             horizontalalignment='right', verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=1))

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'FINAL-Full-Mean-Correlation-vs-Spatial-Distance-btw-Neurons.png'))
    plt.savefig(os.path.join(path_clustering_data, 'FINAL-Full-Mean-Correlation-vs-Spatial-Distance-btw-Neurons.svg'), format='svg')

plt.show()

In [ ]:
# Saving FULL dataset
if save_figure:
    data_to_save = pd.DataFrame({
        'Distance (micrometers)': bins_center,
        'Mean Correlation Coefficient': mean_correlations_per_bin,
        'Standard Error of Correlation': stderr_correlations_per_bin
    })

    # Specify a file path to save the data
    data_file_path = os.path.join(path_clustering_data, 'full_correlation_vs_distance_data.csv')

    # Save the DataFrame to a CSV file
    data_to_save.to_csv(data_file_path, index=False)

In [ ]:
# Plotting until 1000 micrometers and "filtering" parameters so that we truly analyze from 0 to 1000 
bins_center = (distance_bins[:-1] + distance_bins[1:]) / 2  # The bin centers

# Filter data to include only distances up to 1000 micrometers
filter_mask = bins_center <= 1000
filtered_bins_center = bins_center[filter_mask]
filtered_mean_correlations = np.array(mean_correlations_per_bin)[filter_mask]
filtered_stderr_correlations = np.array(stderr_correlations_per_bin)[filter_mask]  # Apply the same filter to stderr

plt.figure(figsize=(10, 6))
plt.errorbar(filtered_bins_center, filtered_mean_correlations, yerr=filtered_stderr_correlations, fmt='o', ecolor='darkgray', elinewidth=3, capsize=5, linestyle='-', linewidth=2)

plt.title('Mean Correlation vs. Spatial Distance Between Neurons \n (using equally spaced binning)')
plt.xlabel('Distance (μm)')
plt.ylabel('Mean Correlation Coefficient')
plt.grid(True)

# Set x-axis limits to extend to 1000 micrometers
plt.xlim(0, 1000)

# Annotate bin size in micrometers
bin_size_micrometers = (bins_center[1] - bins_center[0])
bin_size_micrometers = max(round(bin_size_micrometers), 1) # round to nearest whole number
micrometer_label = "μm" if bin_size_micrometers <= 1 else "μm"
plt.annotate(f'Bin Size: {round(bin_size_micrometers, 2)} {micrometer_label}', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=10,
             horizontalalignment='right', verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=1))

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'FINAL-Segmented-Mean-Correlation-vs-Spatial-Distance-btw-Neurons.png'))
    plt.savefig(os.path.join(path_clustering_data, 'FINAL-Segmented-Mean-Correlation-vs-Spatial-Distance-btw-Neurons.svg'), format='svg')
    
plt.show()


In [ ]:
# Saving SEGMENTED dataset from 0 to 1000 micrometers
if save_figure:
    data_to_save = pd.DataFrame({
        'Distance (micrometers)': filtered_bins_center,
        'Mean Correlation Coefficient': filtered_mean_correlations,
        'Standard Error of Correlation': filtered_stderr_correlations
    })

    # Specify a file path to save the data
    data_file_path = os.path.join(path_clustering_data, '1000um_correlation_vs_distance_data.csv')

    # Save the filtered DataFrame to a CSV file
    data_to_save.to_csv(data_file_path, index=False)

### Fitting a 2nd degree polynomial and saving its coefficients to later compare them across organoids/conditions

In [ ]:
bins_center = (distance_bins[:-1] + distance_bins[1:]) / 2  # The bin centers

# Filter data to include only distances up to 1000 micrometers
filter_mask = bins_center <= 1000
filtered_bins_center = bins_center[filter_mask]
filtered_mean_correlations = np.array(mean_correlations_per_bin)[filter_mask]
filtered_stderr_correlations = np.array(stderr_correlations_per_bin)[filter_mask]  # Apply the same filter to stderr

plt.figure(figsize=(10, 6))
plt.errorbar(filtered_bins_center, filtered_mean_correlations, yerr=filtered_stderr_correlations, fmt='o', ecolor='darkgray', elinewidth=3, capsize=5, linestyle='-', linewidth=2)

# Choose the order of the polynomial (2 for quadratic, 3 for cubic)
polynomial_order = 3

# Fit the polynomial to the filtered data
poly_coefficients = np.polyfit(filtered_bins_center, filtered_mean_correlations, polynomial_order)

# Create a polynomial function from the coefficients
poly_function = np.poly1d(poly_coefficients)

# Generate a smooth set of x-values for plotting the polynomial curve
smooth_x = np.linspace(0, 1000, 500)

# Calculate the corresponding y-values using the polynomial function
smooth_y = poly_function(smooth_x)

# Determine the correct suffix for the polynomial order
if polynomial_order == 2:
    order_suffix = "nd"
elif polynomial_order == 3:
    order_suffix = "rd"
else:
    order_suffix = "th"  # Default for other cases

# Plot the polynomial curve with dynamic label
plt.plot(smooth_x, smooth_y, label=f'{polynomial_order}{order_suffix} order polynomial fit')

plt.title('Fitting a Polynomial Curve to \n Mean Correlation vs. Spatial Distance Between Neurons \n (using equally spaced binning)')
plt.xlabel('Distance (μm)')
plt.ylabel('Mean Correlation Coefficient')
plt.grid(True)

# Set x-axis limits to extend to 1000 micrometers
plt.xlim(0, 1000)

# Annotate bin size in micrometers
bin_size_micrometers = (bins_center[1] - bins_center[0])
bin_size_micrometers = max(round(bin_size_micrometers), 1) # round to nearest whole number
micrometer_label = "μm" if bin_size_micrometers <= 1 else "μm"
plt.annotate(f'Bin Size: {round(bin_size_micrometers, 2)} {micrometer_label}', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=10,
             horizontalalignment='right', verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=1))

# Customize legend to match the annotation style
legend = plt.legend(loc='upper right', bbox_to_anchor=(0.965, 0.90),  # Adjust location as needed
                    frameon=True, framealpha=1, edgecolor='black', facecolor='white')
legend.get_frame().set_linewidth(1)

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'FINAL-FittingPoly-Segmented-Mean-Correlation-vs-Spatial-Distance-btw-Neurons.png'))

# Save the figure as SVG
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'FINAL-FittingPoly-Segmented-Mean-Correlation-vs-Spatial-Distance-btw-Neurons.svg'), format='svg')
    
plt.show()

In [ ]:
# Saving polynomial coefficients of the fitted curve
if save_figure:
    # Create a DataFrame from the coefficients
    coefficients_df = pd.DataFrame({
        'Order': range(polynomial_order, -1, -1),  # Orders from highest to lowest
        'Coefficient': poly_coefficients
    })

    # Specify a file path to save the data
    data_file_path = os.path.join(path_clustering_data, 'polynomial_coefficients.csv')

    # Save the filtered DataFrame to a CSV file
    coefficients_df.to_csv(data_file_path, index=False)

In [ ]:
# Bin size is set dynamically instead of manually

# Load deconvolved fluorescence signal data depending on the condition being analyzed
if run == 'run':
    num_rows = matlab_data['F_dff_dec'].shape[0]  # Get the number of rows
    neuronal_signal_data = matlab_data['F_dff_dec'][0:min(6600, num_rows), :] # takes only the first 5min 30sec of spont. activity, as standardized in our experimental scheme 
elif run in ['run_1', 'run_2', 'run_3']:
    neuronal_signal_data = matlab_data['F_dff_dec_segmented']
elif run in ['run_4', 'run_5']:
    neuronal_signal_data = matlab_data['F_dff_dec_subset']
else:
    print("Invalid run specified")
    neuronal_signal_data = None  # or some default value

# Calculate pairwise correlations
num_neurons = neuronal_signal_data.shape[1]
correlation_matrix = np.zeros((num_neurons, num_neurons))
for i in range(num_neurons):
    for j in range(i+1, num_neurons):
        corr, _ = pearsonr(neuronal_signal_data[:, i], neuronal_signal_data[:, j])
        correlation_matrix[i, j] = corr
        correlation_matrix[j, i] = corr

# Calculate pairwise Euclidean distances between neurons
spatial_coordinates = labeled_neurons[:, :2]
distances_matrix = squareform(pdist(spatial_coordinates)) * conversion_factor # converting distances to micrometers

# Define distance bins and calculate mean correlations for each bin
max_distance = np.max(distances_matrix)
distance_bins = np.linspace(0, max_distance, 20)
distance_bin_indices = np.digitize(distances_matrix, distance_bins)

mean_correlations_per_bin = []
stderr_correlations_per_bin = []  # Use standard error (not standard deviation)

for i in range(1, len(distance_bins)):
    indices = np.where(distance_bin_indices == i)
    mean_corr = np.mean(correlation_matrix[indices])
    
    # Number of observations in the bin
    num_observations = len(correlation_matrix[indices])
    
    # Calculate standard deviation
    std_dev_corr = np.std(correlation_matrix[indices])
    
    # Calculate standard error: standard deviation divided by the square root of the number of observations
    if num_observations > 0:
        stderr_corr = std_dev_corr / np.sqrt(num_observations)
    else:
        stderr_corr = np.nan  # Assign NaN if no observations in the bin

    mean_correlations_per_bin.append(mean_corr)
    stderr_correlations_per_bin.append(stderr_corr)

# Plotting
bins_center = (distance_bins[:-1] + distance_bins[1:]) / 2
plt.figure(figsize=(10, 6))
plt.errorbar(bins_center, mean_correlations_per_bin, yerr=stderr_correlations_per_bin, fmt='o', ecolor='darkgray', elinewidth=3, capsize=5, linestyle='-', linewidth=2)
plt.title('Mean Correlation vs. Spatial Distance Between Neurons \n (using equally spaced binning that is dynamically defined)')
plt.xlabel('Distance (μm)')
plt.ylabel('Mean Correlation Coefficient')
plt.grid(True)

# Annotate bin size in micrometers
bin_size_micrometers = (bins_center[1] - bins_center[0])
bin_size_micrometers = max(round(bin_size_micrometers), 1) # round to nearest whole number
micrometer_label = "μm" if bin_size_micrometers <= 1 else "μm"
plt.annotate(f'Bin Size: {round(bin_size_micrometers, 2)} {micrometer_label}', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=10,
             horizontalalignment='right', verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=1))

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Updated-DynamicBinning-Full-Mean-Correlation-vs-Spatial-Distance-btw-Neurons.png'))
    
plt.show()


The analysis of how correlation coefficients vary with spatial distances between neurons has been completed. We categorized the distances into bins and calculated the mean correlation for each bin. The results show that the mean correlation tends to decrease as the distance between neurons increases, which is a common pattern in spatially structured data. This visual representation can help in understanding the spatial organization and connectivity of neurons based on their activity patterns.

#### Identify the top 10 neuron pairs with the highest correlations 

In [ ]:
# Flatten the upper triangle of the correlation matrix to avoid duplicates
upper_tri_corr = correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)]

# Find the indices of the top ten correlations
top_indices = np.argsort(upper_tri_corr)[-10:][::-1]  # Sort and reverse to get top values

# Extract the corresponding neuron pairs
neuron_pairs_top_correlations = np.array(np.triu_indices_from(correlation_matrix, k=1)).T[top_indices]

# Display information about these neuron pairs including cluster assignments
for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    corr_value = correlation_matrix[neuron1, neuron2]
    coord1, coord2 = spatial_coordinates[neuron1], spatial_coordinates[neuron2]
    cluster1, cluster2 = labeled_neurons[neuron1, 2], labeled_neurons[neuron2, 2]
    print(f"Pair {i+1}: Neuron {neuron1} (Cluster {cluster1}) and Neuron {neuron2} (Cluster {cluster2})")
    print(f"   Correlation: {corr_value}")
    print(f"   Coordinates: Neuron {neuron1} - {coord1}, Neuron {neuron2} - {coord2}\n")


In [ ]:
# Prepare data for saving
data_for_saving = []
for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    corr_value = correlation_matrix[neuron1, neuron2]
    coord1, coord2 = spatial_coordinates[neuron1], spatial_coordinates[neuron2]
    cluster1, cluster2 = labeled_neurons[neuron1, 2], labeled_neurons[neuron2, 2]
    data_for_saving.append([neuron1, neuron2, corr_value, cluster1, cluster2, *coord1, *coord2])

# Convert to a DataFrame
df = pd.DataFrame(data_for_saving, columns=['Neuron1', 'Neuron2', 'Correlation', 'Cluster1', 'Cluster2', 'X1', 'Y1', 'X2', 'Y2'])

# Save to CSV if save_figure is True
if save_figure:
    csv_filename = os.path.join(path_clustering_data, 'top_correlations.csv')
    df.to_csv(csv_filename, index=False)
    

#### Spatial locations of neurons with top 10 correlations

In [ ]:
plt.figure(figsize=(10, 8))

# Display the background image
plt.imshow(Cn, cmap='gray')

# Overlay the scatter plot for all neurons
scatter = plt.scatter(spatial_coordinates[:, 0], spatial_coordinates[:, 1], 
                      c=labeled_neurons[:, 2], cmap=custom_colormap, 
                      s=100, alpha=0.6, edgecolor=edge_colors)

# Plot the top five neuron pairs with the highest correlations
for pair_index in neuron_pairs_top_correlations:
    neuron1, neuron2 = pair_index[0], pair_index[1]
    plt.plot([spatial_coordinates[neuron1, 0], spatial_coordinates[neuron2, 0]], 
             [spatial_coordinates[neuron1, 1], spatial_coordinates[neuron2, 1]], 
             'ro-', linewidth=2, markersize=5)  # Red lines for top neuron pairs

# Create legend for clusters
adjusted_labels = ['Not Assigned' if cluster == 0 else f'Cluster {int(cluster)}' for cluster in unique_clusters]
legend_patches = [mpatches.Patch(color=custom_colormap(cluster / (len(unique_clusters) - 1 if cluster != 0 else 1)), label=label) 
                  for cluster, label in sorted(zip(unique_clusters, adjusted_labels))]

# Add legend entry for top correlations using a line symbol
top_corr_line = mlines.Line2D([], [], color='red', marker='o', markersize=5, label='Top Correlations', linestyle='-')
legend_patches.append(top_corr_line)

plt.legend(handles=legend_patches, loc='upper left')
plt.title('Spatial Locations of Neurons with Top 10 Correlations')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Spatial-Locations-of-Neurons-with-Top-Correlations.png'))
    
plt.show()


#### Plot the neural activity signal (i.e. deconvolved fluorescence signal) of neurons with top 10 correlations

In [ ]:
frame_rate = 20  # frames per second

# Flatten the upper triangle of the correlation matrix to avoid duplicates
upper_tri_corr = correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)]

# Find the indices of the top ten correlations
top_indices = np.argsort(upper_tri_corr)[-10:][::-1]  # Sort and reverse to get top values

# Extract the corresponding neuron pairs
neuron_pairs_top_correlations = np.array(np.triu_indices_from(correlation_matrix, k=1)).T[top_indices]

# Number of frames
num_frames = neuronal_signal_data.shape[0]
time_seconds = np.arange(num_frames) / frame_rate  # Convert frame numbers to time in seconds

# Plotting the neuronal activity for these pairs
fig, axes = plt.subplots(10, 2, figsize=(15, 20))  # 10 rows, 2 columns

for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    # Plot neuron1 activity
    axes[i, 0].plot(time_seconds, neuronal_signal_data[:, neuron1] * 100)  # Multiply by 100 for %ΔF/F
    axes[i, 0].set_title(f'Neuron {neuron1} Activity')

    # Plot neuron2 activity
    axes[i, 1].plot(time_seconds, neuronal_signal_data[:, neuron2] * 100)  # Multiply by 100 for %ΔF/F
    axes[i, 1].set_title(f'Neuron {neuron2} Activity')

    # Consistently annotate all neuron pair labels
    axes[i, 0].annotate(f'Neuron Pair #{i+1}', xy=(-0.10, 0.5), xycoords='axes fraction', 
                        fontsize=12, ha='right', va='center', rotation=90)

# Set global labels for the entire figure
fig.text(0.5, 0.01, 'Time (seconds)', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, '% ΔF/F', ha='center', va='center', rotation='vertical', fontsize=20)

# Add a main title for the overall plot
fig.suptitle('Deconvolved Fluorescence Signal of the Top Ten Neuron Pairs', fontsize=23)

# Adjust the layout to prevent overlap
fig.tight_layout(rect=[0.03, 0.03, 0.97, 0.97])

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Deconvolved-Fluorescence-Signal-of-Top-Ten-Neuron-Pairs.png'))

plt.show()


## 5) Spectral analysis
Explore the frequency domain of the signals to reveal any oscillatory patterns that might not be evident in the time domain.

In [ ]:
# Load FULL (i.e. not segmented/subset) deconvolved fluorescence signal data, keeping in mind the recodring durations of each run
# For spectral analysis, we are no longer using the F_dff_dec_segmented or F_dff_dec_subset signal 
# (i.e. the ones that contain only the post-stimulation windows or just a subset of the full dataset)

def slice_data(data, max_row):
    num_rows = data.shape[0]  # Get the number of rows
    return data[0:max_row, :] if num_rows >= max_row else data

if run == 'run':
    neuronal_signal_data = slice_data(matlab_data['F_dff_dec'], 6600) # 5min30sec
elif run in ['run_1', 'run_2', 'run_3']:
    if run == 'run_1':
        neuronal_signal_data = slice_data(matlab_data['F_dff_dec'], 6600)
    else:
        neuronal_signal_data = slice_data(matlab_data['F_dff_dec'], 12600) # 10min30sec
elif run in ['run_4', 'run_5']:
    neuronal_signal_data = slice_data(matlab_data['F_dff_dec'], 12600)
else:
    print("Invalid run specified")
    neuronal_signal_data = None  # or some default value

# The slice_data function checks if the number of columns in the data is greater than or equal to the maximum column index specified (max_col). If so, it slices the data up to that column; otherwise, it returns the data as is.
# This function is then used in your if-elif-else structure to handle the slicing based on the run value.

In [ ]:
# Complete Deconvolved Fluorescence Signal of the Top Ten Neuron Pairs 
# (as opposed to the plot we did earlier, which was with the segmented or subset data)

# Number of frames
num_frames = neuronal_signal_data.shape[0]
time_seconds = np.arange(num_frames) / frame_rate  # Convert frame numbers to time in seconds

# Plotting the neuronal activity for these pairs
fig, axes = plt.subplots(10, 2, figsize=(15, 20))  # 10 rows, 2 columns

for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    # Plot neuron1 activity
    axes[i, 0].plot(time_seconds, neuronal_signal_data[:, neuron1] * 100)  # Multiply by 100 for %ΔF/F
    axes[i, 0].set_title(f'Neuron {neuron1} Activity')

    # Plot neuron2 activity
    axes[i, 1].plot(time_seconds, neuronal_signal_data[:, neuron2] * 100)  # Multiply by 100 for %ΔF/F
    axes[i, 1].set_title(f'Neuron {neuron2} Activity')

    # Consistently annotate all neuron pair labels
    axes[i, 0].annotate(f'Neuron Pair #{i+1}', xy=(-0.10, 0.5), xycoords='axes fraction', 
                        fontsize=12, ha='right', va='center', rotation=90)

# Set global labels for the entire figure
fig.text(0.5, 0.01, 'Time (seconds)', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, '% ΔF/F', ha='center', va='center', rotation='vertical', fontsize=20)

# Add a main title for the overall plot
fig.suptitle('Complete Deconvolved Fluorescence Signal of the Top Ten Neuron Pairs', fontsize=23)

# Adjust the layout to prevent overlap
fig.tight_layout(rect=[0.03, 0.03, 0.97, 0.97])

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Complete-Deconvolved-Fluorescence-Signal-of-Top-Ten-Neuron-Pairs.png'))

plt.show()

In [ ]:
neuronal_signal_data.shape

### Welch's Method

In [ ]:
# Function to calculate the Power Spectral Density (PSD) of a signal
def plot_psd(signal, ax, title, fs=20):
    """
    Calculate and plot the Power Spectral Density (PSD) of a signal on a given axes.
    
    :param signal: Input signal
    :param ax: Axes to plot on
    :param title: Title for the plot
    :param fs: Sampling frequency
    """
    freqs, psd = welch(signal, fs=fs)
    ax.semilogy(freqs, psd)
    ax.set_title(title)
    # ax.set_xlabel('Frequency')
    # ax.set_ylabel('Power')
    ax.grid(True)

# Spectral analysis for each neuron in the top pairs
fig, axes = plt.subplots(10, 2, figsize=(15, 20))

fs = 20  # Define the sampling frequency if known
for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    signal1 = neuronal_signal_data[:, neuron1]
    signal2 = neuronal_signal_data[:, neuron2]

    plot_psd(signal1, axes[i, 0], f'Neuron {neuron1} PSD')
    plot_psd(signal2, axes[i, 1], f'Neuron {neuron2} PSD')

    # Consistently annotate all neuron pair labels
    axes[i, 0].annotate(f'Neuron Pair #{i+1}', xy=(-0.10, 0.5), xycoords='axes fraction', 
                        fontsize=12, ha='right', va='center', rotation=90)

# Set global labels for the entire figure
fig.text(0.5, 0.01, 'Frequency (Hz)', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, 'Power (a.u.²/Hz)', ha='center', va='center', rotation='vertical', fontsize=20)

# Adjust the layout to prevent overlap and add a main title for the overall plot
fig.suptitle('Power Spectral Density (PSD) of Each Neuron in the Top Ten Pairs \n (using Welch Method)', fontsize=23)
fig.tight_layout(rect=[0.03, 0.03, 0.97, 0.97])

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Power-Spectral-Density-of-Each-Neuron-in-Top-Ten-Pairs.png'))

plt.show()


### Wavelet Transform

Without specific parameters, "cmor" uses default values for its bandwidth and center frequency, which might not be optimal for all types of data. Therefore, I changed the wavelet_name parameter in the plot_cwt function to 'cmor1.5-1.0'. This specifies a Continuous Morlet Wavelet with a bandwidth frequency of 1.5 and a center frequency of 1.0. You can adjust these values (B and C in cmorB-C) to experiment with different wavelet properties. By specifying these parameters, you tailor the wavelet to better suit the specific characteristics of your data, like the frequency range of interest.

In [ ]:
# Scale on y-axis: Often used for detailed analysis where the relationship between the wavelet scales and the signal is of interest.

def plot_cwt(signal, ax, title, scales, wavelet_name='cmor1.5-1.0', frame_rate=20):
    """
    Perform and plot the Continuous Wavelet Transform (CWT) of a signal.
    
    :param signal: Input signal
    :param ax: Axes to plot on
    :param title: Title for the plot
    :param scales: Array of scales for the CWT
    :param wavelet_name: Name of the wavelet to use
    :param frame_rate: Frame rate for converting frame numbers to time
    """
    coefficients, frequencies = pywt.cwt(signal, scales, wavelet_name)
    # Convert frame numbers to time in seconds
    time_seconds = np.arange(len(signal)) / frame_rate
    ax.imshow(np.abs(coefficients), extent=[0, time_seconds[-1], 1, max(scales)], cmap='jet', aspect='auto')
    ax.set_title(title)
    # ax.set_xlabel('Time (seconds)')
    # ax.set_ylabel('Scale (a.u.)')
    ax.invert_yaxis()

# Define scales for the CWT
scales = np.arange(1, 128)

# Plot CWT for each neuron in the top pairs
fig, axes = plt.subplots(10, 2, figsize=(15, 20))

for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    signal1 = neuronal_signal_data[:, neuron1]
    signal2 = neuronal_signal_data[:, neuron2]

    plot_cwt(signal1, axes[i, 0], f'Neuron {neuron1} CWT', scales)
    plot_cwt(signal2, axes[i, 1], f'Neuron {neuron2} CWT', scales)

    axes[i, 0].annotate(f'Neuron Pair #{i+1}', xy=(-0.10, 0.5), xycoords='axes fraction', 
                        fontsize=12, ha='right', va='center', rotation=90)

# Add global labels and main title
fig.text(0.5, 0.01, 'Time (seconds)', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, 'Scale (a.u.)', ha='center', va='center', rotation='vertical', fontsize=20)
fig.suptitle('Continuous Wavelet Transform (CWT) of Each Neuron in the Top Ten Pairs', fontsize=23)

fig.tight_layout(rect=[0.03, 0.03, 0.97, 0.97])

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Scale-Continuous-Wavelet-Transform-of-Each-Neuron-in-Top-Ten-Pairs.png'))

plt.show()


#### Frequency intead of Scale on the Y-Axis

In the original CWT implementation, the y-axis represented the scale, which is a standard way of depicting wavelet transform results. However, interpreting scales can be less intuitive because they are inversely related to frequency (i.e., a higher scale corresponds to a lower frequency and vice versa). Adjusting the y-axis to display frequency instead of scale makes the interpretation more straightforward:

1) Direct Frequency Representation:
Displaying frequency directly on the y-axis aligns with common expectations and interpretations, especially for those familiar with spectral analysis. It's easier to relate the findings to known frequency bands (like alpha, beta, etc., in neural data).

2) Conversion from Scale to Frequency:
The conversion from scale to frequency is done based on the properties of the chosen wavelet. For the Morlet wavelet, this conversion takes into account its center frequency and bandwidth to map scales to corresponding frequencies.

----

Important to know:

1. Frequency Range Analysis:

The maximum frequency you can analyze is dictated by the Nyquist frequency, which is half of the sampling frequency. For a sampling rate of 20 Hz, the Nyquist frequency is 10 Hz. This means you can analyze frequencies up to 10 Hz. Any frequency component above 10 Hz cannot be reliably detected due to the sampling rate, and attempting to analyze these frequencies may lead to aliasing.

2. Choice of Scales:

The scales in the CWT should be chosen to cover the frequency range up to the Nyquist frequency. Since the data is sampled at 20 Hz, and neural activity of interest often occurs in lower frequency bands, you should select scales that correspond to the frequency range of 0-10 Hz.

3. Wavelet Parameters:

The wavelet's center frequency and bandwidth should be chosen considering the sampling rate to ensure the wavelet can capture the frequency components present in the data. For a 20 Hz sampling rate, a wavelet that focuses on lower frequencies (like the Morlet wavelet) would be suitable.

4. Temporal Resolution:

The sampling frequency also affects the temporal resolution of the analysis. With a 20 Hz sampling rate, each frame represents a 50ms interval. The CWT analysis will have this temporal granularity.

In [ ]:
# Frequency on y-axis: More commonly used for spectral analysis where the focus is on identifying the frequency components of the signal over time.

def plot_cwt(signal, ax, title, fs, wavelet_name='cmor1.5-1.0'):
    """
    Perform and plot the Continuous Wavelet Transform (CWT) of a signal.
    
    :param signal: Input signal
    :param ax: Axes to plot on
    :param title: Title for the plot
    :param fs: Sampling frequency
    :param wavelet_name: Name of the wavelet to use
    """
    # Define the frequency range for CWT
    max_freq = fs / 2
    min_freq = 0.1  # Adjust as needed for your data
    freqs = np.linspace(min_freq, max_freq, num=100)
    
    # Convert to scales
    scales = pywt.scale2frequency(wavelet_name, [1 / freq for freq in freqs]) * fs
    
    # Perform CWT
    coefficients, frequencies = pywt.cwt(signal, scales, wavelet_name, sampling_period=1/fs)
    
    # Convert frame numbers to time in seconds
    time_seconds = np.arange(len(signal)) / fs
    ax.imshow(np.abs(coefficients), extent=[0, time_seconds[-1], min_freq, max_freq], cmap='jet', aspect='auto', interpolation='nearest')
    ax.set_title(title)
    # ax.set_xlabel('Time (seconds)')
    # ax.set_ylabel('Frequency (Hz)')
    ax.invert_yaxis()
    
    return coefficients, frequencies


fs = 20  # Sampling frequency

# Plot CWT for each neuron in the top pairs
fig, axes = plt.subplots(10, 2, figsize=(15, 20))

cwt_top10 = {}  # Dictionary to store coefficients to save them later

for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    signal1 = neuronal_signal_data[:, neuron1]
    signal2 = neuronal_signal_data[:, neuron2]

    coefficients1, frequencies1 = plot_cwt(signal1, axes[i, 0], f'Neuron {neuron1} CWT', fs)
    coefficients2, frequencies2 = plot_cwt(signal2, axes[i, 1], f'Neuron {neuron2} CWT', fs)

    # Store the coefficients in the dictionary
    cwt_top10[f'Neuron_{neuron1}'] = (coefficients1, frequencies1)
    cwt_top10[f'Neuron_{neuron2}'] = (coefficients2, frequencies2)

    axes[i, 0].annotate(f'Neuron Pair #{i+1}', xy=(-0.10, 0.5), xycoords='axes fraction', 
                        fontsize=12, ha='right', va='center', rotation=90)

# Add global labels and main title
fig.text(0.5, 0.01, 'Time (seconds)', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, 'Frequency (Hz)', ha='center', va='center', rotation='vertical', fontsize=20)
fig.suptitle('Continuous Wavelet Transform (CWT) of Each Neuron in the Top Ten Pairs', fontsize=23)

fig.tight_layout(rect=[0.03, 0.03, 0.97, 0.97])

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Freq-Continuous-Wavelet-Transform-of-Each-Neuron-in-Top-Ten-Pairs.png'))
    plt.savefig(os.path.join(path_clustering_data, 'Freq-Continuous-Wavelet-Transform-of-Each-Neuron-in-Top-Ten-Pairs.svg'), format='svg')

plt.show()

# Save cwt_top10
if save_figure:
    data_file_path = os.path.join(path_clustering_data, 'cwt_top10.npy')
    np.save(data_file_path, cwt_top10)

# Save as CSV
if save_figure:
    for neuron, (coefficients, frequencies) in cwt_top10.items():
        # Create a DataFrame
        df = pd.DataFrame(coefficients, index=frequencies, columns=np.arange(coefficients.shape[1]) / fs)
        # Save to CSV
        csv_file_path = os.path.join(path_clustering_data, f'{neuron}_cwt.csv')
        df.to_csv(csv_file_path)

# Each CSV file corresponds to one neuron's CWT coefficients.
# The rows in these CSV files represent different frequencies (contained in the frequencies array).
# The columns correspond to different time points (frame numbers converted to seconds).

### Continuous Wavelet Transform (CWT) analysis on entire dataset

In [ ]:
def perform_cwt(signal, scales, wavelet_name='cmor1.5-1.0', fs=20):
    """
    Perform the Continuous Wavelet Transform (CWT) on a given signal.
    
    :param signal: The input signal (one-dimensional array).
    :param scales: Scales for the CWT.
    :param wavelet_name: Name of the wavelet to use.
    :param fs: Sampling frequency.
    :return: CWT coefficients and corresponding frequencies.
    """
    coefficients, frequencies = pywt.cwt(signal, scales, wavelet_name, sampling_period=1/fs)
    return coefficients, frequencies


num_neurons = neuronal_signal_data.shape[1]
fs = 20  # Sampling frequency

# Define frequency range for CWT
max_freq = fs / 2
min_freq = 0.1  # You can adjust this based on your data
freqs = np.linspace(min_freq, max_freq, num=100)

# Convert frequencies to scales
scales = pywt.scale2frequency('cmor1.5-1.0', [1 / freq for freq in freqs]) * fs

# Dictionary to store CWT results
cwt_allresults = {}

for neuron in range(num_neurons):
    signal = neuronal_signal_data[:, neuron]
    coeffs, freqs = perform_cwt(signal, scales, fs=fs)
    cwt_allresults[neuron] = {
        'coefficients': coeffs,
        'frequencies': freqs
    }

# Save CWT results
if save_figure:
    data_file_path = os.path.join(path_clustering_data, 'cwt_allresults.npy')
    np.save(data_file_path, cwt_allresults)

# Save ALL RESULTS as CSV (each neuron's results are saved as a separate file)
# if save_figure:
    # for neuron, data in cwt_allresults.items():
        # Extract coefficients and frequencies
        # coeffs = data['coefficients']
        # freqs = data['frequencies']

        # Create a DataFrame
        # Assuming coeffs is a 2D array (frequencies x time)
        # and freqs is a 1D array (frequencies)
        # df = pd.DataFrame(coeffs, index=freqs, columns=np.arange(coeffs.shape[1]) / fs)

        # Save to CSV
        # csv_file_path = os.path.join(path_clustering_data, f'neuron_{neuron}_cwt.csv')
        # df.to_csv(csv_file_path)


#### Average CWT Coefficients

In [ ]:
frame_rate = 20  # frames per second

# Assuming cwt_allresults contains the CWT coefficients for each neuron
cwt_avg = np.mean([np.abs(cwt_allresults[neuron]['coefficients']) for neuron in cwt_allresults], axis=0)

# Convert frame numbers to time in seconds
num_frames = cwt_avg.shape[1]
time_seconds = np.arange(num_frames) / frame_rate

plt.figure(figsize=(10, 6))
plt.imshow(cwt_avg, aspect='auto', extent=[0, time_seconds[-1], min_freq, max_freq], cmap='jet')
plt.title('Average CWT Coefficients Across All Neurons')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.colorbar(label='Magnitude')

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Average-Continuous-Wavelet-Transform-Coefficients-Across-All-Neurons.png'))
    plt.savefig(os.path.join(path_clustering_data, 'Average-Continuous-Wavelet-Transform-Coefficients-Across-All-Neurons.svg'), format='svg')

plt.show()


# Save cwt_avg
if save_figure:
    data_file_path = os.path.join(path_clustering_data, 'cwt_avg.npy')
    np.save(data_file_path, cwt_avg)

# Save as CSV
if save_figure:
    # Assuming freqs contains the frequencies corresponding to the rows of cwt_avg
    df_cwt_avg = pd.DataFrame(cwt_avg, index=freqs, columns=time_seconds)
    # Specify the path for the CSV file
    csv_file_path = os.path.join(path_clustering_data, 'cwt_avg.csv')
    # Save to CSV
    df_cwt_avg.to_csv(csv_file_path)
    
# The rows of the CSV file correspond to different frequencies (contained in the freqs array).
# The columns represent different time points, calculated as frame numbers converted to seconds (time_seconds array).


### 3D representations of CWT data

#### For average CWT coefficients

In [ ]:
# Assuming cwt_allresults contains the CWT coefficients for each neuron
cwt_avg = np.mean([np.abs(cwt_allresults[neuron]['coefficients']) for neuron in cwt_allresults], axis=0)

# Prepare the time-frequency grid
fs = 20  # Sampling frequency
n_timepoints = cwt_avg.shape[1]
time = np.linspace(0, n_timepoints / fs, n_timepoints)
max_freq = fs / 2
min_freq = 0.1  # Adjust as needed
freqs = np.linspace(min_freq, max_freq, num=cwt_avg.shape[0])
T, F = np.meshgrid(time, freqs)

# Create a 3D plot
fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface
surf = ax.plot_surface(T, F, cwt_avg, cmap='jet', edgecolor='none')
# viridis or jet

# Labels and titles
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.set_title('Average CWT Coefficients Across All Neurons')
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Frequency (Hz)')
ax.set_zlabel('Magnitude')

# Adding a color bar with adjusted padding
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=8, pad=0.05)  # Increase pad to move colorbar to the right

# Adjust the z-axis label position
ax.set_zlabel('Magnitude', labelpad=8)  # Increase the labelpad value as needed

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(left=0.05, right=0.85)  # Adjust these parameters

# Save the figure with bbox_inches='tight' to reduce white space
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Opt1-3D-Continuous-Wavelet-Transform-Coefficients-Average.png'), bbox_inches='tight')
    plt.savefig(os.path.join(path_clustering_data, 'Opt1-3D-Continuous-Wavelet-Transform-Coefficients-Average.svg'), format='svg', bbox_inches='tight')

plt.show()

In [ ]:
# Line Plots on a 3D Surface
# Instead of using a filled surface, you can plot lines on a 3D graph. This approach uses the same time-frequency grid but plots lines for each frequency.

fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(111, projection='3d')

# Plotting lines for each frequency
for i, freq in enumerate(freqs):
    ax.plot(time, [freq] * len(time), cwt_avg[i, :], color=plt.cm.jet(i / len(freqs)))

# Labels and titles
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.set_title('Average CWT Coefficients Across All Neurons')
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Frequency (Hz)')
ax.set_zlabel('Magnitude')

# Adding a color bar with adjusted padding
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=8, pad=0.05)  # Increase pad to move colorbar to the right

# Adjust the z-axis label position
ax.set_zlabel('Magnitude', labelpad=8)  # Increase the labelpad value as needed

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(left=0.05, right=0.85)  # Adjust these parameters

# Save the figure with bbox_inches='tight' to reduce white space
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, 'Opt2-3D-Continuous-Wavelet-Transform-Coefficients-Average.png'), bbox_inches='tight')
    plt.savefig(os.path.join(path_clustering_data, 'Opt2-3D-Continuous-Wavelet-Transform-Coefficients-Average.svg'), format='svg', bbox_inches='tight')

plt.show()

#### For top 10 neuron pairs with highest correlations

In [ ]:
# Define frequency range and scales for CWT (common for all neurons)
max_freq = fs / 2
min_freq = 0.1  # Adjust as needed
freqs = np.linspace(min_freq, max_freq, num=100)
scales = pywt.scale2frequency('cmor1.5-1.0', [1 / freq for freq in freqs]) * fs

n_pairs = 10

# Create a figure with subplots
fig = plt.figure(figsize=(12, n_pairs * 8))

for i, (neuron1, neuron2) in enumerate(neuron_pairs_top_correlations):
    # Prepare the time-frequency grid
    n_timepoints = neuronal_signal_data.shape[0]
    time = np.linspace(0, n_timepoints / fs, n_timepoints)
    T, F = np.meshgrid(time, freqs)
    
    # Perform CWT for neuron1 and neuron2
    coefficients1, _ = pywt.cwt(neuronal_signal_data[:, neuron1], scales, 'cmor1.5-1.0', sampling_period=1/fs)
    coefficients2, _ = pywt.cwt(neuronal_signal_data[:, neuron2], scales, 'cmor1.5-1.0', sampling_period=1/fs)

    # Plot for neuron1
    ax1 = fig.add_subplot(n_pairs, 2, 2*i+1, projection='3d')
    # ax1.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    # ax1.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax1.zaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax1.plot_surface(T, F, np.abs(coefficients1), cmap='jet', edgecolor='none')
    ax1.set_title(f'Neuron {neuron1} CWT', pad=10)  # Adjust title spacing with the pad parameter
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Frequency (Hz)')
    ax1.set_zlabel('Magnitude')
    ax1.text2D(-0.15, 0.5, f'Neuron Pair #{i+1}', transform=ax1.transAxes, fontsize=12, ha='center', va='center', rotation=90)

    # Plot for neuron2
    ax2 = fig.add_subplot(n_pairs, 2, 2*i+2, projection='3d')
    # ax2.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    # ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax2.zaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax2.plot_surface(T, F, np.abs(coefficients2), cmap='jet', edgecolor='none')
    ax2.set_title(f'Neuron {neuron2} CWT', pad=10)  # Adjust title spacing with the pad parameter
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Frequency (Hz)')
    ax2.set_zlabel('Magnitude')

plt.tight_layout()
plt.subplots_adjust(top=1)  # Reduce the top margin to make space for the title

# Add a main title for the overall plot, adjust y position
# fig.suptitle('CWT Coefficients of Each Neuron in the Top Ten Pairs', fontsize=23, y=1)

# Adjust the spacing
plt.subplots_adjust(wspace=0.4, hspace=0.4, right=0.9)  # Adjust 'right' parameter as needed

# Save the figure
if save_figure:
    plt.savefig(os.path.join(path_clustering_data, '3D-Continuous-Wavelet-Transform-Coefficients-Top-10-Neurons.png'))

plt.show()